# Import Libraries

In [1]:
from dash import Dash, html, dcc, callback, Output, Input, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import json

# Initialize App 

In [2]:
app = Dash(
    __name__,
    external_stylesheets=[
        dbc.themes.BOOTSTRAP,
        dbc.icons.FONT_AWESOME
    ]
)

# Example Dataset and Choropleth Figure

In [3]:
df = pd.read_csv("aid_data.csv")
df = df.dropna()

# Dataframe for sum of donations given grouped by country
donors_df = df.rename(columns={"donor":"country"}).sort_values(by=["commitment_amount_usd_constant"], ascending=True).groupby(["country"])["commitment_amount_usd_constant"].sum().reset_index(name="total_donated")

# Dataframe for sum of donations received grouped by country
recipients_df = df.rename(columns={"recipient":"country"}).groupby(["country"])["commitment_amount_usd_constant"].sum().reset_index(name="total_received")

# Dataframe for net donations grouped by country
combined_df = pd.merge(donors_df, recipients_df, how="outer", on="country").fillna(0)
combined_df["net"] = combined_df["total_donated"] - combined_df["total_received"]

# Choropleth figure
fig_q1 = px.choropleth(combined_df,
                        title="Net Donations by Country",
                        locations="country",
                        locationmode="country names",
                        color="net",
                        color_continuous_scale="PiYg",
                        hover_name="country",
                        range_color=[-4e6, 4e6],
                        projection="natural earth")

df

,aiddata_id,year,donor,recipient,commitment_amount_usd_constant,coalesced_purpose_code,coalesced_purpose_name
0,12191891,1998,Australia,Indonesia,1.033230e+05,24040,Informal/semi-formal fin. intermed.
1,27339565,2003,Australia,Singapore,2.979690e+02,15105,"Government and civil society, purpose unspecif..."
2,34378730,2007,Australia,Colombia,1.395560e+05,15130,Legal and judicial development
3,13308187,1999,Australia,Timor-Leste,2.271530e+04,12220,Basic health care
4,38835178,2008,Australia,"Bilateral, unspecified",4.468550e+04,15110,Public sector policy and adm. management
...,...,...,...,...,...,...,...
494,34085698,2007,United States,"Bilateral, unspecified",1.422430e+07,13020,Reproductive health care
495,26038958,2005,United States,Brazil,3.255560e+05,41020,Biosphere protection
496,50081282,2009,United States,Pakistan,1.761800e+05,43010,Multisector aid
497,50064569,2009,United States,Ukraine,5.495000e+04,15130,Legal and judicial development


# Dash Layout

In [4]:
fig_q1.update_layout(clickmode='event+select')

country = None

app.layout = html.Div([
    dcc.Graph(
        id="test-map",
        figure=fig_q1
    ),
    # html.H1(id='hover-data', children=[
    #     "Test Header"
    # ]),
    dbc.Modal(
        [
            dbc.ModalHeader(dbc.ModalTitle(id="modal-text", children=[
                "Placeholder"
            ]))
        ],
        id="modal-sm",
        size="sm",
        is_open=False,
    )
])

# @callback(
#     Output('hover-data', 'children'),
#     Input('test-map', 'clickData')
# )
# def display_click_data(clickData):
#     # return json.dumps(clickData, indent=2)["points"][0]
#     if clickData:
#         country = clickData["points"][0]["location"]
#         return clickData["points"][0]["location"]
#     country = None
#     return None

@callback([
        Output("modal-text", "children"),
        Output("modal-sm", "is_open")
    ],[
        Input("test-map", "clickData")
    ],[
        State("modal-sm", "is_open")
    ]
)
def update_modal(clickData, is_open):
    print(clickData)
    if clickData:
        return clickData["points"][0]["location"], not is_open
    return None, is_open
    # return clickData

# def toggle_modal(n1, is_open):
#     if n1:
#         print(is_open)
#         return not is_open
#     print(is_open)
#     return is_open

# app.callback(
#     Output("modal-sm", "is_open"),
#     Input("open-sm", "n_clicks"),
#     State("modal-sm", "is_open"),
# )(toggle_modal)

# Run Dash App

In [5]:
app.run(debug=True)

None
None
None
{'points': [{'curveNumber': 0, 'pointNumber': 144, 'pointIndex': 144, 'location': 'United States', 'z': 167538652.872, 'hovertext': 'United States', 'bbox': {'x0': 570.4151801187745, 'x1': 570.4151801187745, 'y0': 168.8515057951438, 'y1': 168.8515057951438}}]}
{'points': [{'curveNumber': 0, 'pointNumber': 130, 'pointIndex': 130, 'location': 'Sudan', 'z': -2412554.16, 'hovertext': 'Sudan', 'bbox': {'x0': 745.0550848630422, 'x1': 745.0550848630422, 'y0': 208.29048330773415, 'y1': 208.29048330773415}}]}
{'points': [{'curveNumber': 0, 'pointNumber': 65, 'pointIndex': 65, 'location': 'India', 'z': -22872540.209999997, 'hovertext': 'India', 'bbox': {'x0': 814.4607375104691, 'x1': 814.4607375104691, 'y0': 196.65632506339023, 'y1': 196.65632506339023}}]}
{'points': [{'curveNumber': 0, 'pointNumber': 9, 'pointIndex': 9, 'location': 'Australia', 'z': 1383428.869, 'hovertext': 'Australia', 'bbox': {'x0': 890.3508980518174, 'x1': 890.3508980518174, 'y0': 277.760343614425, 'y1': 277.